# House Price Prediction with LightGBM

<a href="https://colab.research.google.com/github/VertaAI/examples/blob/main/deployment/lightgbm/lightgbm-house-price-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Dependencies

This notebook has been tested with **Python 3.8.16** and the following package versions:

In [ ]:
%%capture
!pip install lightgbm==2.2.3
!pip install scikit-learn==1.0.2
!pip install verta==0.21.1
!pip install wget==3.2

## 2. Imports

In [ ]:
import cloudpickle
import lightgbm as lgb
import os
import pandas as pd
import wget

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from verta import Client
from verta.environment import Python
from verta.registry import VertaModelBase, verify_io
from verta.utils import ModelAPI

## 3. Model Training

### 3.1 Load Training Data

In [ ]:
file_name = 'melb-data.csv'

if not os.path.isfile(file_name):
    wget.download(f"http://s3.amazonaws.com/verta-starter/{file_name}")

df = pd.read_csv(file_name)

In [ ]:
df = df.dropna(axis=0)

In [ ]:
target_col = 'Price'
feature_cols = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 'Distance', 'YearBuilt', 'Car', 'Propertycount']

In [ ]:
X = df.loc[:,feature_cols]
y = df.loc[:,target_col]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)

In [ ]:
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

### 3.2 Train/Test Code

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [ ]:
gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round = 200,
    valid_sets = lgb_eval,
    callbacks = [lgb.early_stopping(stopping_rounds = 5)]
)

In [ ]:
y_pred = gbm.predict(X_test)

In [ ]:
MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = mean_squared_error(y_test, y_pred, squared=False) 

In [ ]:
print(f"MAE: {round(MAE, 3)}")
print(f"MSE: {round(MSE, 3)}")
print(f"RMSE: {round(RMSE, 3)}")

## 4. Verta Set Up

In [ ]:
os.environ['VERTA_HOST'] = ''
os.environ['VERTA_EMAIL'] = ''
os.environ['VERTA_DEV_KEY'] = ''

In [ ]:
client = Client(os.environ['VERTA_HOST'])

## 5. Model Register

In [ ]:
with open('model.pkl', 'wb') as f:
    cloudpickle.dump(gbm, f)

In [ ]:
class Predictor(VertaModelBase):
    def __init__(self, artifacts):
        self.model = cloudpickle.load(open(artifacts['serialized_model'], 'rb'))
    
    @verify_io
    def predict(self, data):
        return self.model.predict(data).tolist()

    def describe(self):
        return {
            'method': 'predict',
            'args': f"{self.example()}",
            'returns': '[0]',
            'description': 'Predicts house prices based on LightGBM trained model.',
            'input_description': 'List of house information, such as number of rooms, building area, land size, etc.',
            'output_description': 'House price prediction.'
        }
        
    def example(self):
        return [[3.0, 1.0, 206.0, 110.0, 8.4, 1980.0, 1.0, 8801.0]]

In [ ]:
artifacts = {'serialized_model': 'model.pkl'}

In [ ]:
registered_model = client.get_or_create_registered_model(name='House Price Prediction')

In [ ]:
model = registered_model.create_standard_model(
    name = 'lightgbm',
    model_cls = Predictor,
    environment = Python(requirements=['scikit-learn', 'lightgbm']),
    model_api = ModelAPI(X_train, y_train),
    artifacts = artifacts
)

## 6. Model Endpoint

In [ ]:
endpoint = client.get_or_create_endpoint('house-price-prediction-lightgbm')

In [ ]:
endpoint.update(model, wait=True)

In [ ]:
deployed_model = endpoint.get_deployed_model()

## 7. Predictions

In [ ]:
sample_data = X_test.values.tolist()[:5]

In [ ]:
results = deployed_model.predict(sample_data)

In [ ]:
df = pd.DataFrame(data=sample_data, columns=X_test.columns)

In [ ]:
df['PricePrediction'] = [round(i, 2) for i in results]

In [ ]:
df

In [ ]:
endpoint.delete()